In [15]:
from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet
from nltk import pos_tag
from nltk.corpus import stopwords
import string
from nltk.tokenize import sent_tokenize, word_tokenize
from sklearn.feature_extraction.text import CountVectorizer
import pandas as pd
import numpy as np
from sklearn.svm import SVC

In [2]:
#load training data
df_training = pd.read_csv(r'G:\Coding ninjas\Data Science & Machine Learning\Milestone 10\Project Twitter Sentiment Analysis\training_twitter_x_y_train.csv')
df_training

,tweet_id,airline_sentiment,airline,airline_sentiment_gold,name,negativereason_gold,retweet_count,text,tweet_coord,tweet_created,tweet_location,user_timezone
0,567900433542488064,negative,Southwest,NaN,ColeyGirouard,NaN,0,"@SouthwestAir I am scheduled for the morning, ...",NaN,2015-02-17 20:16:29 -0800,Washington D.C.,Atlantic Time (Canada)
1,569989168903819264,positive,Southwest,NaN,WalterFaddoul,NaN,0,@SouthwestAir seeing your workers time in and ...,NaN,2015-02-23 14:36:22 -0800,"Indianapolis, Indiana; USA",Central Time (US & Canada)
2,568089179520954368,positive,United,NaN,LocalKyle,NaN,0,@united Flew ORD to Miami and back and had gr...,NaN,2015-02-18 08:46:29 -0800,Illinois,Central Time (US & Canada)
3,568928195581513728,negative,Southwest,NaN,amccarthy19,NaN,0,@SouthwestAir @dultch97 that's horse radish 😤🐴,NaN,2015-02-20 16:20:26 -0800,NaN,Atlantic Time (Canada)
4,568594180014014464,negative,United,NaN,J_Okayy,NaN,0,@united so our flight into ORD was delayed bec...,NaN,2015-02-19 18:13:11 -0800,NaN,Eastern Time (US & Canada)
...,...,...,...,...,...,...,...,...,...,...,...,...
10975,569934458364813313,neutral,American,NaN,Cottopanama85,NaN,0,@AmericanAir followback,NaN,2015-02-23 10:58:58 -0800,"ohio,panama",NaN
10976,568564006329434113,positive,United,NaN,PaulBEsteves,NaN,0,@united thanks for the help. Wish the phone re...,NaN,2015-02-19 16:13:17 -0800,Brooklyn,Eastern Time (US & Canada)
10977,569643648910028801,negative,US Airways,NaN,runfixsteve,NaN,0,@usairways the. Worst. Ever. #dca #customerser...,NaN,2015-02-22 15:43:24 -0800,"St. Augustine, Florida",NaN
10978,568864981917110272,negative,US Airways,NaN,CLChicosky,NaN,0,@nrhodes85: look! Another apology. DO NOT FLY ...,NaN,2015-02-20 12:09:15 -0800,NaN,NaN


In [3]:
#drop columns which are not needed
df_training.drop('tweet_id', axis = 1, inplace = True)
df_training.drop('airline', axis = 1, inplace = True)
df_training.drop('airline_sentiment_gold', axis = 1, inplace = True)
df_training.drop('name', axis = 1, inplace = True)
df_training.drop('negativereason_gold', axis = 1, inplace = True)
df_training.drop('retweet_count', axis = 1, inplace = True)
df_training.drop('tweet_coord', axis = 1, inplace = True)
df_training.drop('tweet_created', axis = 1, inplace = True)
df_training.drop('tweet_location', axis = 1, inplace = True)
df_training.drop('user_timezone', axis = 1, inplace = True)
df_training

,airline_sentiment,text
0,negative,"@SouthwestAir I am scheduled for the morning, ..."
1,positive,@SouthwestAir seeing your workers time in and ...
2,positive,@united Flew ORD to Miami and back and had gr...
3,negative,@SouthwestAir @dultch97 that's horse radish 😤🐴
4,negative,@united so our flight into ORD was delayed bec...
...,...,...
10975,neutral,@AmericanAir followback
10976,positive,@united thanks for the help. Wish the phone re...
10977,negative,@usairways the. Worst. Ever. #dca #customerser...
10978,negative,@nrhodes85: look! Another apology. DO NOT FLY ...


In [17]:
X  = df_training['text'].values
X = X.astype(str)
X

array(['@SouthwestAir I am scheduled for the morning, 2 days after the fact, yes..not sure why my evening flight was the only one Cancelled Flightled',
       '@SouthwestAir seeing your workers time in and time out going above and beyond is why I love flying with you guys. Thank you!',
       '@united Flew ORD to Miami and back and  had great crew, service on both legs. THANKS',
       ..., '@usairways the. Worst. Ever. #dca #customerservice',
       '@nrhodes85: look! Another apology. DO NOT FLY @USAirways',
       '@united you are by far the worst airline. 4 plane delays on 1 round trip flight. How is that possible.'],
      dtype='<U186')

In [21]:
#convert each comment into list of words
Xd = [word_tokenize(i) for i in X]
Xd[0]


['@',
 'SouthwestAir',
 'I',
 'am',
 'scheduled',
 'for',
 'the',
 'morning',
 ',',
 '2',
 'days',
 'after',
 'the',
 'fact',
 ',',
 'yes',
 '..',
 'not',
 'sure',
 'why',
 'my',
 'evening',
 'flight',
 'was',
 'the',
 'only',
 'one',
 'Cancelled',
 'Flightled']

In [32]:

#load stop words
stop = set(stopwords.words('english'))

#load punctuations
punct = list(string.punctuation)

#load integers
inte = [i for i in range(10)]

#update set stop + punct
stop.update(punct)

#update set stop + inte
stop.update(inte)


In [33]:
#clean data & lemmatize

lemmatizer = WordNetLemmatizer()


def get_simple_pos(tag):
    if tag.startswith('J'):
        return wordnet.ADJ
    
    elif tag.startswith('N'):
        return wordnet.NOUN
    
    elif tag.startswith('V'):
        return wordnet.VERB
    
    elif tag.startswith('R'):
        return wordnet.ADV
    
    else:
        return wordnet.NOUN

def clean(words):
    output = []
    
    for w in words:
        if w.lower() not in stop:  #if word is not in stop, then only proceed further
            
            pos = pos_tag([w])          #find pos tag
            
            #lemmatize the word & append it in new array
            clean_word = lemmatizer.lemmatize(w, pos = get_simple_pos(pos[0][1])) 
            output.append(clean_word.lower())
            
    return output

    
            

In [34]:
#clean the training data
Xc = [clean(comment) for comment in Xd]

In [35]:
Xc[0]

['southwestair',
 'schedule',
 'morning',
 '2',
 'day',
 'fact',
 'yes',
 '..',
 'sure',
 'even',
 'flight',
 'one',
 'cancelled',
 'flightled']

In [43]:
#join the list of words & convert it into one single string
Xtrain = [' '.join(i) for i in Xc]
Xtrain[0]

'southwestair schedule morning 2 day fact yes .. sure even flight one cancelled flightled'

In [36]:
# similarly load testing data & perform all operations

df_testing = pd.read_csv(r'G:\Coding ninjas\Data Science & Machine Learning\Milestone 10\Project Twitter Sentiment Analysis\test_twitter_x_test.csv')
df_testing.drop('tweet_id', axis = 1, inplace = True)
df_testing.drop('airline', axis = 1, inplace = True)
df_testing.drop('airline_sentiment_gold', axis = 1, inplace = True)
df_testing.drop('name', axis = 1, inplace = True)
df_testing.drop('negativereason_gold', axis = 1, inplace = True)
df_testing.drop('retweet_count', axis = 1, inplace = True)
df_testing.drop('tweet_coord', axis = 1, inplace = True)
df_testing.drop('tweet_created', axis = 1, inplace = True)
df_testing.drop('tweet_location', axis = 1, inplace = True)
df_testing.drop('user_timezone', axis = 1, inplace = True)
df_testing

,text
0,@AmericanAir In car gng to DFW. Pulled over 1h...
1,"@AmericanAir after all, the plane didn’t land ..."
2,@SouthwestAir can't believe how many paying cu...
3,@USAirways I can legitimately say that I would...
4,@AmericanAir still no response from AA. great ...
...,...
3655,@USAirways Been stuck for 40+ minutes due to l...
3656,@USAirways 4 hours... 4 hours... FOUR HOURS. ...
3657,Nice RT @VirginAmerica: The man of steel might...
3658,@AmericanAir Aww Thanks AA..DFW was on GMA up ...


In [40]:
Xte = df_testing['text'].values
Xte = Xte.astype(str)
Xd_te = [word_tokenize(i) for i in Xte]
Xd_te[0]

['@',
 'AmericanAir',
 'In',
 'car',
 'gng',
 'to',
 'DFW',
 '.',
 'Pulled',
 'over',
 '1hr',
 'ago',
 '-',
 'very',
 'icy',
 'roads',
 '.',
 'On-hold',
 'with',
 'AA',
 'since',
 '1hr',
 '.',
 'Ca',
 "n't",
 'reach',
 'arpt',
 'for',
 'AA2450',
 '.',
 'Wat',
 '2',
 'do',
 '?']

In [41]:
#clean the testing data
Xc_te = [clean(comment) for comment in Xd_te]

In [42]:
Xc_te[0]

['americanair',
 'car',
 'gng',
 'dfw',
 'pulled',
 '1hr',
 'ago',
 'icy',
 'road',
 'on-hold',
 'aa',
 'since',
 '1hr',
 'ca',
 "n't",
 'reach',
 'arpt',
 'aa2450',
 'wat',
 '2']

In [44]:
#join the list of words & convert it into one single string
Xtest = [' '.join(i) for i in Xc_te]
Xtest[0]

"americanair car gng dfw pulled 1hr ago icy road on-hold aa since 1hr ca n't reach arpt aa2450 wat 2"

In [45]:
#load countVectorizer & convert training & testing data

vec = CountVectorizer()
Xtrain_v = vec.fit_transform(Xtrain)
Xtrain_v.toarray()

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=int64)

In [47]:
Xtest_v = vec.transform(Xtest)
Xtest_v.toarray()

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=int64)

In [48]:
Ytrain = df_training['airline_sentiment'].values
Ytrain

array(['negative', 'positive', 'positive', ..., 'negative', 'negative',
       'negative'], dtype=object)

In [50]:
# apply SVC
svc = SVC()
svc.fit(Xtrain_v, Ytrain)

SVC()

In [51]:
#predicting output for testing data
Ypred = svc.predict(Xtest_v)
Ypred

array(['negative', 'negative', 'negative', ..., 'neutral', 'positive',
       'negative'], dtype=object)

In [55]:
Ypred = pd.Series(Ypred)
Ypred

0       negative
1       negative
2       negative
3       negative
4       positive
          ...   
3655    negative
3656    negative
3657     neutral
3658    positive
3659    negative
Length: 3660, dtype: object

In [58]:
Ypred.to_csv('Ypred twitter.csv', index_label = False)

In [57]:
svc.score(Xtrain_v, Ytrain)

0.9173952641165756